In [1]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
import math
from ROOT import RooFit
from ROOT import TChain
from ROOT import RooStats
from datetime import datetime
from uncertainties import ufloat
from scipy.optimize import curve_fit

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
print(r.gROOT.GetVersion())

%jsroot on

c = r.TCanvas()

6.37.01


In [2]:
# Open a ROOT file to the for branches and events
file = r.TFile.Open("2024_MC_B2OC_UP_B5.root")  # Replace with the actual file name

# Access the TTree (adjust name if needed)
tree = file.Get("ST-b2oc")  # or "ST-b2cc" depending on your dataset

# List all branch names
branches = tree.GetListOfBranches()
for branch in branches:
    print(branch.GetName())
print(branches.GetEntries())
#if tree:
#    n_entries = min(100, tree.GetEntries())
#    for i in range(n_entries):
#        tree.GetEntry(i)
#        print(f"Event {i}: block =", getattr(tree, "block", None))

OSError: Failed to open file 2024_MC_B2OC_UP_B5.root

Error in <TFile::TFile>: file C:\Users\paolo\Coding_projects\2024\2024_MC_B2OC_UP_B5.root does not exist


# Fill Fitting 

In [3]:
import ROOT as r

# List of files to clean
file_list = [
    "2024_B2OC_UP_B5_F10059.root","2024_B2OC_UP_B5_F10061.root", "2024_B2OC_UP_B5_F10066.root", "2024_B2OC_UP_B5_F10069.root",
    "2024_B2OC_UP_B5_F10070.root", "2024_B2OC_UP_B5_F10072.root", "2024_B2OC_UP_B5_F10073.root",
    "2024_B2OC_UP_B5_F10074.root", "2024_B2OC_UP_B5_F10075.root","2024_B2OC_UP_B5_F10077.root","2024_B2OC_UP_B5_F10082.root", "2024_B2OC_UP_B5_F10084.root",
    "2024_B2OC_UP_B5_F10086.root", "2024_B2OC_UP_B5_F10087.root", "2024_B2OC_UP_B5_F10090.root",
    "2024_B2OC_UP_B5_F10091.root", "2024_B2OC_UP_B5_F10092.root", "2024_B2OC_UP_B5_F10093.root", "2024_B2OC_UP_B5_F10094.root", "2024_B2OC_UP_B5_F10095.root",
    "2024_B2OC_UP_B5_F10096.root", "2024_B2OC_UP_B5_F10097.root", "2024_B2OC_UP_B5_F10098.root",
    "2024_B2OC_UP_B5_F10099.root", "2024_B2OC_UP_B5_F10100.root", "2024_B2OC_DOWN_B6_F10104.root", "2024_B2OC_DOWN_B6_F10105.root", "2024_B2OC_DOWN_B6_F10107.root",
    "2024_B2OC_DOWN_B6_F10112.root", "2024_B2OC_DOWN_B6_F10115.root", "2024_B2OC_DOWN_B6_F10116.root", "2024_B2OC_DOWN_B6_F10118.root", "2024_B2OC_DOWN_B6_F10122.root",
    "2024_B2OC_DOWN_B6_F10125.root", "2024_B2OC_DOWN_B6_F10126.root", "2024_B2OC_DOWN_B6_F10127.root",
    "2024_B2OC_DOWN_B6_F10130.root", "2024_B2OC_DOWN_B6_F10132.root", "2024_B2OC_DOWN_B6_F10134.root",
    "2024_B2OC_DOWN_B6_F10137.root", "2024_B2OC_DOWN_B6_F10138.root", "2024_B2OC_DOWN_B7_F10197.root", "2024_B2OC_DOWN_B7_F10199.root", "2024_B2OC_DOWN_B7_F10200.root",
    "2024_B2OC_DOWN_B7_F10201.root", "2024_B2OC_DOWN_B7_F10204.root", "2024_B2OC_DOWN_B7_F10206.root",
    "2024_B2OC_DOWN_B7_F10207.root", "2024_B2OC_DOWN_B7_F10208.root", "2024_B2OC_DOWN_B7_F10209.root",
    "2024_B2OC_DOWN_B7_F10210.root", "2024_B2OC_DOWN_B7_F10211.root", "2024_B2OC_DOWN_B7_F10213.root", "2024_B2OC_UP_B8_F10216.root",
    "2024_B2OC_UP_B8_F10218.root", "2024_B2OC_UP_B8_F10219.root", "2024_B2OC_UP_B8_F10222.root",
    "2024_B2OC_UP_B8_F10223.root", "2024_B2OC_UP_B8_F10225.root", "2024_B2OC_UP_B8_F10226.root",
    "2024_B2OC_UP_B8_F10230.root", "2024_B2OC_UP_B8_F10232.root"
    # Add more files as needed
]

for file_name in file_list:
    print(f"\n--- Processing {file_name} ---")
    
    try:
        f = r.TFile.Open(file_name, "UPDATE")
        if not f or f.IsZombie():
            print(f"Error: Could not open {file_name}")
            continue
            
        # Delete fit_results tree if it exists
        if f.Get("fit_results"):
            f.Delete("fit_results;*")
            print("Deleted 'fit_results' tree.")
        else:
            print("'fit_results' tree not found.")

        # Collect all keys to delete (canvas objects starting with dpi_mass_fit_)
        to_delete = []
        for key in f.GetListOfKeys():
            name = key.GetName()
            if name.startswith("dpi_mass_fit_"):
                to_delete.append(name)

        # Delete them
        for name in to_delete:
            f.Delete(f"{name};*")
            print(f"Deleted '{name}'")

        if len(to_delete) == 0:
            print("No canvas objects starting with 'dpi_mass_fit_' found.")

        f.Close()
        print(f"Finished processing {file_name}")
        
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

print(f"\nCompleted processing {len(file_list)} files.")


--- Processing 2024_B2OC_UP_B5_F10059.root ---
'fit_results' tree not found.
No canvas objects starting with 'dpi_mass_fit_' found.
Finished processing 2024_B2OC_UP_B5_F10059.root

--- Processing 2024_B2OC_UP_B5_F10061.root ---
'fit_results' tree not found.
No canvas objects starting with 'dpi_mass_fit_' found.
Finished processing 2024_B2OC_UP_B5_F10061.root

--- Processing 2024_B2OC_UP_B5_F10066.root ---
'fit_results' tree not found.
No canvas objects starting with 'dpi_mass_fit_' found.
Finished processing 2024_B2OC_UP_B5_F10066.root

--- Processing 2024_B2OC_UP_B5_F10069.root ---
'fit_results' tree not found.
No canvas objects starting with 'dpi_mass_fit_' found.
Finished processing 2024_B2OC_UP_B5_F10069.root

--- Processing 2024_B2OC_UP_B5_F10070.root ---
'fit_results' tree not found.
No canvas objects starting with 'dpi_mass_fit_' found.
Finished processing 2024_B2OC_UP_B5_F10070.root

--- Processing 2024_B2OC_UP_B5_F10072.root ---
'fit_results' tree not found.
No canvas objects

In [ ]:
    "2024_B2OC_UP_B5_F10059.root","2024_B2OC_UP_B5_F10061.root", "2024_B2OC_UP_B5_F10066.root", "2024_B2OC_UP_B5_F10069.root",
    "2024_B2OC_UP_B5_F10070.root", "2024_B2OC_UP_B5_F10072.root", "2024_B2OC_UP_B5_F10073.root",
    "2024_B2OC_UP_B5_F10074.root", "2024_B2OC_UP_B5_F10075.root","2024_B2OC_UP_B5_F10077.root","2024_B2OC_UP_B5_F10082.root", "2024_B2OC_UP_B5_F10084.root",
    "2024_B2OC_UP_B5_F10086.root", "2024_B2OC_UP_B5_F10087.root", "2024_B2OC_UP_B5_F10090.root",
    "2024_B2OC_UP_B5_F10091.root", "2024_B2OC_UP_B5_F10092.root", "2024_B2OC_UP_B5_F10093.root", "2024_B2OC_UP_B5_F10094.root", "2024_B2OC_UP_B5_F10095.root",
    "2024_B2OC_UP_B5_F10096.root", "2024_B2OC_UP_B5_F10097.root", "2024_B2OC_UP_B5_F10098.root",
    "2024_B2OC_UP_B5_F10099.root", "2024_B2OC_UP_B5_F10100.root", "2024_B2OC_DOWN_B6_F10104.root", "2024_B2OC_DOWN_B6_F10105.root", "2024_B2OC_DOWN_B6_F10107.root",
    "2024_B2OC_DOWN_B6_F10112.root", "2024_B2OC_DOWN_B6_F10115.root", "2024_B2OC_DOWN_B6_F10116.root", "2024_B2OC_DOWN_B6_F10118.root", "2024_B2OC_DOWN_B6_F10122.root",
    "2024_B2OC_DOWN_B6_F10125.root", "2024_B2OC_DOWN_B6_F10126.root", "2024_B2OC_DOWN_B6_F10127.root",
    "2024_B2OC_DOWN_B6_F10130.root", "2024_B2OC_DOWN_B6_F10132.root", "2024_B2OC_DOWN_B6_F10134.root",
    "2024_B2OC_DOWN_B6_F10137.root", "2024_B2OC_DOWN_B6_F10138.root", "2024_B2OC_DOWN_B7_F10197.root", "2024_B2OC_DOWN_B7_F10199.root", "2024_B2OC_DOWN_B7_F10200.root",
    "2024_B2OC_DOWN_B7_F10201.root", "2024_B2OC_DOWN_B7_F10204.root", "2024_B2OC_DOWN_B7_F10206.root",
    "2024_B2OC_DOWN_B7_F10207.root", "2024_B2OC_DOWN_B7_F10208.root", "2024_B2OC_DOWN_B7_F10209.root",
    "2024_B2OC_DOWN_B7_F10210.root", "2024_B2OC_DOWN_B7_F10211.root", "2024_B2OC_DOWN_B7_F10213.root", "2024_B2OC_UP_B8_F10216.root",
    "2024_B2OC_UP_B8_F10218.root", "2024_B2OC_UP_B8_F10219.root", "2024_B2OC_UP_B8_F10222.root",
    "2024_B2OC_UP_B8_F10223.root", "2024_B2OC_UP_B8_F10225.root", "2024_B2OC_UP_B8_F10226.root",
    "2024_B2OC_UP_B8_F10230.root", "2024_B2OC_UP_B8_F10232.root"

In [9]:
from array import array                     # ROOT TTrees need C-style arrays
import re                   
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = ["2024_B2OC_UP_B5_F10059.root","2024_B2OC_UP_B5_F10061.root", "2024_B2OC_UP_B5_F10066.root", "2024_B2OC_UP_B5_F10069.root",
    "2024_B2OC_UP_B5_F10070.root", "2024_B2OC_UP_B5_F10072.root", "2024_B2OC_UP_B5_F10073.root",
    "2024_B2OC_UP_B5_F10074.root", "2024_B2OC_UP_B5_F10075.root","2024_B2OC_UP_B5_F10077.root","2024_B2OC_UP_B5_F10082.root", "2024_B2OC_UP_B5_F10084.root",
    "2024_B2OC_UP_B5_F10086.root", "2024_B2OC_UP_B5_F10087.root", "2024_B2OC_UP_B5_F10090.root",
    "2024_B2OC_UP_B5_F10091.root", "2024_B2OC_UP_B5_F10092.root", "2024_B2OC_UP_B5_F10093.root", "2024_B2OC_UP_B5_F10094.root", "2024_B2OC_UP_B5_F10095.root",
    "2024_B2OC_UP_B5_F10096.root", "2024_B2OC_UP_B5_F10097.root", "2024_B2OC_UP_B5_F10098.root",
    "2024_B2OC_UP_B5_F10099.root", "2024_B2OC_UP_B5_F10100.root"
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(True)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))
    
    match = re.search(r'_B(\d+)_F(\d+)', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match else ""
    title_latex = "B^{+} \\rightarrow \\bar{D}^{0}\\pi^{+}" 

    canvas_name = f"dpi_mass_fit_block{match.group(1)}_fill{match.group(2)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str} {fill_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(ymin * 0.7)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()



▶ fitting fill file 2024_B2OC_UP_B5_F10059.root
entries in tree = 23493

=== fitted yields ===
nsig  = 6779 ± 183
nbkg  = 16715 ± 208
ntot  = 23495 (derived)
0.8097266617168801
background fraction = 0.711452+/-0.006109
fit status = 0, covQual = 3

▶ fitting fill file 2024_B2OC_UP_B5_F10061.root
entries in tree = 71868

=== fitted yields ===
nsig  = 20394 ± 304
nbkg  = 51475 ± 351
ntot  = 71868 (derived)
0.7349655938277795
background fraction = 0.716237+/-0.003332
fit status = 0, covQual = 3

▶ fitting fill file 2024_B2OC_UP_B5_F10066.root
entries in tree = 81014

=== fitted yields ===
nsig  = 23851 ± 328
nbkg  = 57166 ± 375
ntot  = 81017 (derived)
0.7926803333042642
background fraction = 0.705607+/-0.003167
fit status = 0, covQual = 3

▶ fitting fill file 2024_B2OC_UP_B5_F10069.root
entries in tree = 182307

=== fitted yields ===
nsig  = 53553 ± 493
nbkg  = 128751 ± 564
ntot  = 182304 (derived)
0.7264768875409484
background fraction = 0.706242+/-0.002116
fit status = 0, covQual = 3

▶

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 24.5799 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -85870.94366 Edm =       80.69893199 NCalls =     25
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.112102
Info in <Minuit2>: MnHesse Done after 36.0163 ms
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -85870.94366
  Edm           : 19216.6157
  Internal parameters:	[     0.2356918764      0.304692654     0.1388456842    -0.1433475689     0.1433475689      1.477141165]	
  Internal gradient  :	[     -293.9801281     -35.95196704     -186.8788433     -27.50089196      82.50234218      -2389.68069]	
  Internal covariance matrix:
[[     0.40792531      2.2491587   -0.047027491    -0.11918573     0.12635641 -0.00056435478]
 [      2.2491587      12.468798    -0.26062006    -0.65620882     0.69568

In [7]:
import ROOT as r

# ------------------------------------------------------------
file_name = "2024_B2OC_DOWN_B7_F10197.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:

    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  tau   =", res_tree.tau,      "±", res_tree.tau_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("dpi_mass_fit_block7_fill10197"+"_log" )  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2oc : TTree
  ST-b2oc : TTree
  fit_results : TTree
  dpi_mass_fit_block7_fill10197 : TCanvas
  dpi_mass_fit_block7_fill10197_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  tau   = -0.0009064647326068043 ± 4.833296200790205e-05
  nsig   = 49309.12695110101 ± 522.1498393267757
  nbkg   = 145934.5059436938 ± 607.6089420717035
  ntot   = 195243.6328947948 ± 801.1423601298787
  frac_cb_2   = 0.7715632863329127


In [249]:
import ROOT as r

file_name = "2024_B2CC_UP_B5_F10090.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("jpsik_mass_fit_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()

Deleted 'fit_results' tree.
Deleted 'jpsik_mass_fit_block5_fill10090'
Deleted 'jpsik_mass_fit_block5_fill10090_log'


In [2]:
#"2024_B2CC_UP_B5_F10059.root", "2024_B2CC_UP_B5_F10061.root","2024_B2CC_UP_B5_F10066.root", "2024_B2CC_UP_B5_F10069.root","2024_B2CC_UP_B5_F10070.root","2024_B2CC_UP_B5_F10072.root","2024_B2CC_UP_B5_F10073.root",
#    "2024_B2CC_UP_B5_F10074.root", "2024_B2CC_UP_B5_F10075.root", "2024_B2CC_UP_B5_F10077.root","2024_B2CC_UP_B5_F10082.root","2024_B2CC_UP_B5_F10084.root","2024_B2CC_UP_B5_F10086.root","2024_B2CC_UP_B5_F10087.root",
#    "2024_B2CC_UP_B5_F10090.root","2024_B2CC_UP_B5_F10091.root","2024_B2CC_UP_B5_F10092.root","2024_B2CC_UP_B5_F10093.root","2024_B2CC_UP_B5_F10094.root","2024_B2CC_UP_B5_F10095.root","2024_B2CC_UP_B5_F10096.root",
#    "2024_B2CC_UP_B5_F10097.root","2024_B2CC_UP_B5_F10098.root","2024_B2CC_UP_B5_F10099.root","2024_B2CC_UP_B5_F10100.root", "2024_B2CC_DOWN_B6_F10104.root", "2024_B2CC_DOWN_B6_F10105.root", "2024_B2CC_DOWN_B6_F10107.root",
#    "2024_B2CC_DOWN_B6_F10112.root","2024_B2CC_DOWN_B6_F10115.root","2024_B2CC_DOWN_B6_F10116.root","2024_B2CC_DOWN_B6_F10117.root","2024_B2CC_DOWN_B6_F10118.root","2024_B2CC_DOWN_B6_F10122.root","2024_B2CC_DOWN_B6_F10125.root",
#    "2024_B2CC_DOWN_B6_F10126.root","2024_B2CC_DOWN_B6_F10127.root","2024_B2CC_DOWN_B6_F10130.root","2024_B2CC_DOWN_B6_F10132.root","2024_B2CC_DOWN_B6_F10134.root","2024_B2CC_DOWN_B6_F10137.root","2024_B2CC_DOWN_B6_F10138.root",
#    "2024_B2CC_DOWN_B7_F10197.root","2024_B2CC_DOWN_B7_F10199.root","2024_B2CC_DOWN_B7_F10200.root", "2024_B2CC_DOWN_B7_F10201.root","2024_B2CC_DOWN_B7_F10204.root","2024_B2CC_DOWN_B7_F10206.root","2024_B2CC_DOWN_B7_F10207.root",
#    "2024_B2CC_DOWN_B7_F10208.root","2024_B2CC_DOWN_B7_F10209.root","2024_B2CC_DOWN_B7_F10210.root","2024_B2CC_DOWN_B7_F10211.root", "2024_B2CC_DOWN_B7_F10213.root","2024_B2CC_UP_B8_F10214.root","2024_B2CC_UP_B8_F10215.root","2024_B2CC_UP_B8_F10216.root","2024_B2CC_UP_B8_F10218.root","2024_B2CC_UP_B8_F10219.root",
#    "2024_B2CC_UP_B8_F10222.root", "2024_B2CC_UP_B8_F10223.root", "2024_B2CC_UP_B8_F10225.root","2024_B2CC_UP_B8_F10226.root","2024_B2CC_UP_B8_F10230.root","2024_B2CC_UP_B8_F10232.root"

In [250]:
from array import array                     # ROOT TTrees need C-style arrays
import re
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_B2CC_UP_B5_F10090.root"
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)


mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(True)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    match = re.search(r'_B(\d+)_F(\d+)', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match else ""
    title_latex = "B^{+} \\rightarrow J/\psi K^{+}"

    canvas_name = f"jpsik_mass_fit_block{match.group(1)}_fill{match.group(2)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str} {fill_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(ymin * 0.8)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


<>:218: SyntaxWarning: invalid escape sequence '\p'
<>:218: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_16100\51432934.py:218: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting fill file 2024_B2CC_UP_B5_F10090.root
entries in tree = 889838

=== fitted yields ===
nsig  = 68842 ± 592
nbkg  = 821019 ± 1050
ntot  = 889862 (derived)
0.693321694907261
background fraction = 0.922637+/-0.000620
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #0 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5614.05
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5034.06
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5715.31
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5776.51
[#1] INFO:DataHandling -- RooTre

<>:218: SyntaxWarning: invalid escape sequence '\p'
<>:218: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_16100\51432934.py:218: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting fill file 2024_B2CC_UP_B5_F10090.root
entries in tree = 889838

=== fitted yields ===
nsig  = 68842 ± 592
nbkg  = 821019 ± 1050
ntot  = 889862 (derived)
0.693321694907261
background fraction = 0.922637+/-0.000620
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #0 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5614.05
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5034.06
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5715.31
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5776.51
[#1] INFO:DataHandling -- RooTre

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 918.119 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       -6357329.56 Edm =       78992.48912 NCalls =     29
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 1.98317
Info in <Minuit2>: MnHesse Done after 1.21539 s
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -6357329.56
  Edm           : 371602.6736
  Internal parameters:	[     0.2891274493                0     0.1761089065    -0.1433475689     0.1433475689      1.505265445]	
  Internal gradient  :	[     -7165.514057     -3349.552677      806.9944276     -20330.02864      60989.93201     -99024.14467]	
  Internal covariance matrix:
[[    0.059992063     0.11052646 -0.00038381333  -0.0027318983   0.0074888276  6.3274048e-06]
 [     0.11052646     0.20420353 -0.00070946935  -0.0050394884    0.01381451

In [251]:
import ROOT as r

# ------------------------------------------------------------
file_name = "2024_B2CC_UP_B5_F10090.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("jpsik_mass_fit_block5_fill10090" + "_log" )  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  jpsik_mass_fit_block5_fill10090 : TCanvas
  jpsik_mass_fit_block5_fill10090_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5279.347966002943 ± 0.08153721748840326
  nsig   = 68842.40571625112 ± 591.5003062349497
  nbkg   = 821019.242553267 ± 1050.214182834723
  nsig   = 68842.40571625112 ± 591.5003062349497
  frac_cb_2   = 0.693321694907261


# Block Fitting 

In [77]:
import ROOT as r

file_name = "2024_B2OC_DOWN_B6.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("dpi_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()


Deleted 'fit_results' tree.
Deleted 'dpi_mass_fit_block6'
Deleted 'dpi_mass_fit_block6_log'


In [78]:
from array import array                     # ROOT TTrees need C-style arrays
import re                   
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_B2OC_DOWN_B6.root",
    
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = make_branch("status")
        br_covqual        = make_branch("covQual")
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))
    
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow \\bar{D}^{0}\\pi^{+}" 

    canvas_name = f"dpi_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(ymin * 0.7)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()



▶ fitting block file 2024_B2OC_DOWN_B6.root
entries in tree = 2469387

=== fitted yields ===
nsig  = 750057 ± 2880
nbkg  = 1719153 ± 3044
ntot  = 2469209 (derived)
0.6574771952089065
background fraction = 0.696236+/-0.000894
fit status = 3, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5012.17
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4881.53
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5018.38
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4896.35
[#1] INFO:DataHandling -- RooT

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 3.20817 s
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -20519717.38 Edm =       13782.42913 NCalls =     39
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 1.74238
Info in <Minuit2>: MnHesse Done after 4.31054 s
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -20519717.38
  Edm           : 67387.19797
  Internal parameters:	[    -0.3155557491     0.2356918764      0.304692654     0.1388456842    -0.1433475689     0.1433475689      1.477141165]	
  Internal gradient  :	[     -21693.61091     -29724.70333      -4367.62697     -27371.92888     -2718.496994      8155.432076     -319858.8038]	
  Internal covariance matrix:
[[   0.0042120092  -0.0017430306   -0.018010867  8.3866714e-05 -0.00016534782  0.00021465965  2.0111982e-05]
 [  -0.0017430306  0.000729

In [79]:
# ------------------------------------------------------------
file_name = "2024_B2OC_DOWN_B6.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("dpi_mass_fit_block6_log")  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2oc : TTree
  ST-b2oc : TTree
  fit_results : TTree
  dpi_mass_fit_block6 : TCanvas
  dpi_mass_fit_block6_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err
  status
  covQual

entries in the tree: 1

first entry snapshot:
  mean   = 5278.510790903801 ± 0.03346810980565351
  nsig   = 750056.5420006266 ± 2879.7400619719992
  nbkg   = 1719152.503480072 ± 3043.937195828068
  ntot   = 2469209.0454806983 ± 4190.281193031338
  frac_cb_2   = 0.6574771952089065


In [51]:
import ROOT as r

file_name = "2024_B2CC_UP_B8.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("dpi_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()


'fit_results' tree not found.


In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_B2CC_UP_B8.root", "2024_B2CC_UP_B7.root", "2024_B2CC_UP_B8.root"
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)


mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting Block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow J/\psi K^{+}"

    canvas_name = f"jpsik_mass_fit_block{match.group(1)}_fill{match.group(2)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str} {fill_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(ymin * 0.8)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


<>:218: SyntaxWarning: invalid escape sequence '\p'
<>:218: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_18068\1800349468.py:218: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting Block file 2024_B2CC_UP_B8.root
entries in tree = 9113339

=== fitted yields ===
nsig  = 463515 ± 6709
nbkg  = 8656362 ± 1895203
ntot  = 9119877 (derived)
0.09454711005009181
background fraction = 0.949175+/-0.010585
fit status = 3, covQual = 2
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #0 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5100.46
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5615.12
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5761.74
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #5 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5094.87
[#1] INFO:DataHandling -- Ro

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 9.7734 s
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -86071648.63 Edm =       1479270.613 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 7.1447
Info in <Minuit2>: MnHesse Done after 15.5055 s
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -86071648.63
  Edm           : 469874.5564
  Internal parameters:	[    -0.0900014567     0.2891274493                0     0.1761089065    -0.1433475689     0.1433475689      1.505265445]	
  Internal gradient  :	[     -79264.28347     -80529.93329     -38176.67831     -16927.05623     -237195.9208      711586.3601       -1142247.4]	
  Internal covariance matrix:
[[   0.0078155672   -0.002114498  -0.0084358637  7.8031222e-06 -1.0421854e-05  3.9663916e-05  3.0474919e-06]
 [   -0.002114498  0.00057894

In [5]:
# ------------------------------------------------------------
file_name = "2024_B2CC_UP_B8.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("jpsik_mass_fit_block8_fillNone_log")  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  jpsik_mass_fit_block8_fillNone : TCanvas
  jpsik_mass_fit_block8_fillNone_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5279.780322138023 ± 0.008801910325928475
  nsig   = 463515.45459689933 ± 6708.977039461781
  nbkg   = 8656361.775854075 ± 1895202.5391093078
  ntot   = 9119877.230450975 ± 1895214.4138907564
  frac_cb_2   = 0.09454711005009181


In [74]:
import ROOT as r

file_name = "2024_B2CC_DOWN_B6_F10105.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("jpsik_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()


Deleted 'fit_results' tree.
Deleted 'jpsik_mass_fit_block6_fill10105'
Deleted 'jpsik_mass_fit_block6_fill10105_log'


In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_B2CC_DOWN_B6_F10105.root"
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)


mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting Block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))
    
    
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow J/\psi K^{+}"

    canvas_name = f"jpsik_mass_fit_block{match.group(1)}_fill{match.group(2)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str} {fill_str}" if match else f"{fname} mass fit"


    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetYaxis().SetTitle("Candidates / (2.5 MeV/c^{2})")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(ymin * 0.8)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


<>:220: SyntaxWarning: invalid escape sequence '\p'
<>:220: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_18068\2311953038.py:220: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting Block file 2024_B2CC_DOWN_B6_F10105.root
entries in tree = 1169484

=== fitted yields ===
nsig  = 92357 ± 672
nbkg  = 1077161 ± 1199
ntot  = 1169519 (derived)
0.5324671310544481
background fraction = 0.921030+/-0.000535
fit status = 3, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5484.76
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5744.34
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5458.47
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #5 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5586.45
[#1] INFO:DataHandling --

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 1.41166 s
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -8676187.573 Edm =       123211.0839 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 5.69173
Info in <Minuit2>: MnHesse Done after 2.15031 s
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -8676187.573
  Edm           : 18439.92399
  Internal parameters:	[    -0.0900014567     0.2891274493                0     0.1761089065    -0.1433475689     0.1433475689      1.505265445]	
  Internal gradient  :	[      -9446.48179     -9509.185743       -4484.5339      1777.338547     -26556.20201      79668.40316     -121600.6517]	
  Internal covariance matrix:
[[    0.051716308   -0.017964991   -0.065481911  8.5464803e-05 -7.6721985e-06  2.2761904e-05  2.6504162e-05]
 [   -0.017964991   0.00630

In [76]:
# ------------------------------------------------------------
file_name = "2024_B2CC_DOWN_B6_F10105.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("jpsik_mass_fit_block6_fill10105_log")  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  jpsik_mass_fit_block6_fill10105 : TCanvas
  jpsik_mass_fit_block6_fill10105_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5279.325037148736 ± 0.07282126123709531
  nsig   = 92357.21897328968 ± 671.8648738506818
  nbkg   = 1077161.4311956258 ± 1198.6292903397698
  ntot   = 1169518.6501689155 ± 1374.0868183542161
  frac_cb_2   = 0.5324671310544481


# Functions

# Monte Carlo mass fits

## Monte Carlo mass fit, $B^+ \rightarrow \bar{D}^0\pi^+$ 


In [26]:
import ROOT as r

file_name = "2024_MC_B2OC_DOWN_B7.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("dpi_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()


Deleted 'fit_results' tree.
Deleted 'dpi_mc_mass_fit_block7'
Deleted 'dpi_mc_mass_fit_block7_log'


In [27]:
from array import array                     # ROOT TTrees need C-style arrays
import re                   
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_MC_B2OC_DOWN_B7.root",
    
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 4900, 5950)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.5, Nevt * 0.05, Nevt * 1)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.1, Nevt * 0.0, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = make_branch("status")
        br_covqual        = make_branch("covQual")
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background),r.RooFit.FillColor(r.kGreen + 3),r.RooFit.FillStyle(3001),r.RooFit.DrawOption("F"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.LineColor(r.kGreen+2), r.RooFit.LineStyle(r.kDashed))
 
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue)) , r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F")
    
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow \\bar{D}^{0}\\pi^{+}" 

    canvas_name = f"dpi_mc_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+3)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Automatic y-axis minimum for log scale 
    #x_min_bg = 5400 
    #x_max_bg = 5450
    #n_bins = 100
    #bin_width = (x.getMax() - x.getMin()) / n_bins
    #n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    #ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))
#
    # Save log-y version
    frame.SetMinimum(0.3)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()



▶ fitting block file 2024_MC_B2OC_DOWN_B7.root
entries in tree = 59763

=== fitted yields ===
nsig  = 59294 ± 246
nbkg  = 475 ± 36
ntot  = 59769 (derived)
0.7262774299871978
background fraction = 0.007953+/-0.000605
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1863 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4871.36
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3106 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4898.38
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4688 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4888.02
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4793 because Bp_DTF_OwnPV_MASS cannot accommodate the value 6096.33
[#1] INFO:DataHandling -- R

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 66.6395 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -324915.3285 Edm =        8870.63587 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.315893
Info in <Minuit2>: MnHesse Done after 94.4425 ms
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -324915.3285
  Edm           : 7350.98971
  Internal parameters:	[    -0.3155557491     0.2356918764      0.304692654     0.1388456842     -0.909951031   -0.05265590826      1.477141165]	
  Internal gradient  :	[     -1779.921638     -3681.926879      -883.282287     -4684.277411      8317.038908     -25384.80995      592.7171996]	
  Internal covariance matrix:
[[    0.046062002   -0.024919632   -0.060767527  0.00083586465  -0.0040843837  0.00091691214  0.00026121333]
 [   -0.024919632    0.01

In [28]:
# ------------------------------------------------------------
file_name = "2024_MC_B2OC_DOWN_B7.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("dpi_mc_mass_fit_block7_log")  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2oc : TTree
  fit_results : TTree
  dpi_mc_mass_fit_block7 : TCanvas
  dpi_mc_mass_fit_block7_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err
  status
  covQual

entries in the tree: 1

first entry snapshot:
  mean   = 5278.522580086098 ± 0.06494910988021729
  nsig   = 59294.08413221724 ± 246.0059488187544
  nbkg   = 475.32463891999157 ± 36.3804046650782
  ntot   = 59769.408771137234 ± 248.68144421691468
  frac_cb_2   = 0.7262774299871978


## Monte Carlo mass fit,  $B^+ \rightarrow J/\psi K^+$ 


In [23]:
import ROOT as r

file_name = "2024_MC_B2CC_UP_B8.root"  # Change as needed
f = r.TFile.Open(file_name, "UPDATE")
if f.Get("fit_results"):
    f.Delete("fit_results;*")
    print("Deleted 'fit_results' tree.")
else:
    print("'fit_results' tree not found.")

# Collect all keys to delete
to_delete = []
for key in f.GetListOfKeys():
    name = key.GetName()
    if name.startswith("jpsik_"):
        to_delete.append(name)

# Delete them
for name in to_delete:
    f.Delete(f"{name};*")
    print(f"Deleted '{name}'")

f.Close()


'fit_results' tree not found.


In [24]:
from array import array                     # ROOT TTrees need C-style arrays
import re
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    "2024_MC_B2CC_UP_B8.root"
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5050, 5550)    # MeV/c² window

mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)


mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting Block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 1)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.1, Nevt * 0.0, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background),r.RooFit.FillColor(r.kGreen + 3),r.RooFit.FillStyle(3001),r.RooFit.DrawOption("F"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.LineColor(r.kGreen+2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow J/\psi K^{+}"

    canvas_name = f"jpsik_mc_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) adjust as needed
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    ## Automatic y-axis minimum for log scale 
    #x_min_bg = 5400 
    #x_max_bg = 5450
    #n_bins = 100
    #bin_width = (x.getMax() - x.getMin()) / n_bins
    #n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    #ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Save log-y version
    frame.SetMinimum(50)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


<>:219: SyntaxWarning: invalid escape sequence '\p'
<>:219: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_24524\1166642859.py:219: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting Block file 2024_MC_B2CC_UP_B8.root
entries in tree = 244253

=== fitted yields ===
nsig  = 235449 ± 489
nbkg  = 8806 ± 114
ntot  = 244255 (derived)
0.4335132529310381
background fraction = 0.036053+/-0.000454
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #9 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5705.77
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #17 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5666.54
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #20 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5026.24
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #21 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5650.4
[#1] INFO:DataHandling -- RooTreeD

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 262.885 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -1697062.984 Edm =       96930.35125 NCalls =     37
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.71702
Info in <Minuit2>: MnHesse Done after 410.295 ms
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -1697062.984
  Edm           : 58128.56671
  Internal parameters:	[    -0.0900014567     0.2891274493                0     0.1761089065     -0.909951031    -0.6174371036      1.505265445]	
  Internal gradient  :	[      2754.216819      3196.662722       1303.06832      3304.863295      10804.12914     -251697.8479      1782.603091]	
  Internal covariance matrix:
[[    0.014580689   -0.033792313    0.033008262 -0.00042942648  0.00028299104 -6.7340854e-05 -7.9859223e-05]
 [   -0.033792313    0.07

In [25]:
# ------------------------------------------------------------
file_name = "2024_MC_B2CC_UP_B8.root"   #  << pick any file you just fitted
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# ------------------------------------------------------------
# 1) check the fit_results tree and its branches
# ------------------------------------------------------------
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

# ------------------------------------------------------------
# 2) display the saved canvas
# ------------------------------------------------------------
c = f.Get("jpsik_mc_mass_fit_block8_log")  # linear-y canvas
if c:
    c.Draw()                        # will pop up in notebook / X11
else:
    print("canvas object not found; check its name in `f.ls()`")

# if you saved log-y, try:  f.Get("mass_fit_"+file_name+"_log")



--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  jpsik_mc_mass_fit_block8 : TCanvas
  jpsik_mc_mass_fit_block8_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5279.374511136003 ± 0.02111946299783085
  nsig   = 235449.28721855782 ± 489.4580676682963
  nbkg   = 8806.166274551615 ± 113.52247429529962
  ntot   = 244255.45349310944 ± 502.4505469951341
  frac_cb_2   = 0.4335132529310381


## FULL BLOCK TIME STABILITY
